# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
print((document_tree.__dict__))

{'_root': <Element 'mondial' at 0x1045b2910>}


In [4]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [5]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [6]:
import sys
reload(sys)
sys.setdefaultencoding('utf-8')
import pandas as pd

In [7]:
document_tree = ET.parse( './data/mondial_database.xml' )

#### 1. 10 countries with the lowest infant mortality rates

Get the name and infant_mortality subelements from each country element

In [8]:
rate_list = [[country.findtext('name', default = ''), float(country.findtext('infant_mortality', default = 'nan'))]for country in document_tree.findall('country')]    
ratedf = pd.DataFrame(rate_list, columns = ['name', 'infant_mortality'])
ratedf.sort_values(by = 'infant_mortality').head(10)

,name,infant_mortality
38,Monaco,1.81
98,Japan,2.13
117,Bermuda,2.48
36,Norway,2.48
106,Singapore,2.53
37,Sweden,2.60
10,Czech Republic,2.63
78,Hong Kong,2.73
79,Macao,3.13
44,Iceland,3.15


#### 2. 10 cities with the largest population

- Find all cities from all countries
- Find the most recent population estimate based on year for each city

In [9]:
cities = [[city.findtext('name'), float(pop.text),int(pop.attrib['year'])] for city in document_tree.iter('city') for pop in city.iter('population') ]
cities_df = pd.DataFrame(cities, columns = ['city_name', 'population', 'year'])
# http://stackoverflow.com/questions/15705630/python-getting-the-row-which-has-the-max-value-in-groups-using-groupby
idx = cities_df.groupby('city_name')['year'].transform(max) == cities_df['year']
cities_max = cities_df[idx]
cities_max.sort_values('population', ascending=False).head(10)

,city_name,population,year
3750,Shanghai,22315474.0,2010
2607,Istanbul,13710512.0,2012
4303,Mumbai,12442373.0,2011
1546,Moskva,11979529.0,2013
3746,Beijing,11716620.0,2010
8208,São Paulo,11152344.0,2010
3754,Tianjin,11090314.0,2010
3364,Guangzhou,11071424.0,2010
4399,Delhi,11034555.0,2011
3371,Shenzhen,10358381.0,2010


#### 3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

- Get the most recent population estimate for each country
- Get all the ethnic groups for each country
- Merge population dataframe with the ethnic group dataframe by country
- Calculate ethnic group population by country
- Sum ethnic group populations across countries, sort to get top 10


In [10]:
pops = [[country.findtext('name'), float(pop.text), int(pop.attrib['year'])] for country in document_tree.findall('country') for pop in country.findall('population')]
pops_df = pd.DataFrame(pops, columns = ['country_name', 'population', 'year'])
# http://stackoverflow.com/questions/15705630/python-getting-the-row-which-has-the-max-value-in-groups-using-groupby
idx = pops_df.groupby('country_name')['year'].transform(max) == pops_df['year']
pops_max = pops_df[idx]

In [11]:
pops_max.head(10)

,country_name,population,year
8,Albania,2800138.0,2011
23,Greece,10816286.0,2011
34,Macedonia,2059794.0,2011
42,Serbia,7120666.0,2011
52,Montenegro,620029.0,2011
55,Kosovo,1733872.0,2011
63,Andorra,78115.0,2011
73,France,64933400.0,2011
85,Spain,46815916.0,2011
96,Austria,8499759.0,2013


In [12]:
ethnicgroups = [[country.findtext('name'), float(egroup.attrib['percentage']), egroup.text] for country in document_tree.findall('country') for egroup in country.findall('ethnicgroup')]
ethnicgroups_df = pd.DataFrame(ethnicgroups, columns = ['country_name', 'percentage', 'ethnicgroup'])
ethnicgroups_df.head()

,country_name,percentage,ethnicgroup
0,Albania,95.0,Albanian
1,Albania,3.0,Greek
2,Greece,93.0,Greek
3,Macedonia,64.2,Macedonian
4,Macedonia,25.2,Albanian


In [13]:
ethnicgroups_bycountry = pd.merge(pops_max, ethnicgroups_df, on = 'country_name')
ethnicgroups_bycountry.head()
ethnicgroups_bycountry['ethnicgroup_population'] = ethnicgroups_bycountry['population']*ethnicgroups_bycountry['percentage']/100
ethnicgroups_bycountry.groupby('ethnicgroup')['ethnicgroup_population'].sum().sort_values(ascending=False).head(10)


ethnicgroup
Han Chinese    1.245059e+09
Indo-Aryan     8.718156e+08
European       4.948722e+08
African        3.183251e+08
Dravidian      3.027137e+08
Mestizo        1.577344e+08
Bengali        1.467769e+08
Russian        1.318570e+08
Japanese       1.265342e+08
Malay          1.219936e+08
Name: ethnicgroup_population, dtype: float64

#### 4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

Rivers, lakes, and airports are elements of the root.

- Each has a country attribute.
- Each has a name subelement.
- Each has a length, area, and elevation subelement respectively.

In [14]:
rivers = document_tree.findall('river')
river_info = [[river.findtext('name'), river.attrib['country'], float(river.findtext('length', default = 'nan'))] for river in rivers]
river_df = pd.DataFrame(river_info, columns = ['river_name', 'country', 'length']).sort_values('length', ascending = False).head(10)
river_df

,river_name,country,length
174,Amazonas,CO BR PE,6448.0
137,Jangtse,CN,6380.0
136,Hwangho,CN,4845.0
123,Lena,R,4400.0
205,Zaire,RCB ZRE,4374.0
138,Mekong,CN LAO THA K VN,4350.0
115,Irtysch,R KAZ CN,4248.0
186,Niger,RMM RN WAN RG,4184.0
160,Missouri,USA,4130.0
119,Jenissej,R,4092.0


In [15]:
lakes = document_tree.findall('lake')
lake_info = [[lake.findtext('name'), lake.attrib['country'], float(lake.findtext('area', default = 'nan'))] for lake in lakes]
lake_df = pd.DataFrame(lake_info, columns = ['lake_name', 'country', 'area']).sort_values('area', ascending = False).head(10)
lake_df

,lake_name,country,area
54,Caspian Sea,R AZ KAZ IR TM,386400.0
109,Lake Superior,CDN USA,82103.0
81,Lake Victoria,EAT EAK EAU,68870.0
106,Lake Huron,CDN USA,59600.0
108,Lake Michigan,USA,57800.0
47,Dead Sea,IL JOR WEST,41650.0
83,Lake Tanganjika,ZRE Z BI EAT,32893.0
98,Great Bear Lake,CDN,31792.0
43,Ozero Baikal,R,31492.0
89,Lake Malawi,MW MOC EAT,29600.0


In [16]:
airports = document_tree.findall('airport')
airport_info = [[ap.findtext('name'), ap.attrib['country'], ap.findtext('elevation', default = '')] for ap in airports]
airport_df = pd.DataFrame(airport_info, columns = ['airport_name', 'country', 'elevation'])
airport_df.loc[airport_df.elevation == '','elevation'] = 'nan'
airport_df['elevation'] = airport_df['elevation'].astype('float')
airport_df.sort_values('elevation', ascending = False).head(10)

,airport_name,country,elevation
80,El Alto Intl,BOL,4063.0
219,Lhasa-Gonggar,CN,4005.0
241,Yushu Batang,CN,3963.0
813,Juliaca,PE,3827.0
815,Teniente Alejandro Velasco Astete Intl,PE,3311.0
82,Juana Azurduy De Padilla,BOL,2905.0
334,Mariscal Sucre Intl,EC,2813.0
805,Coronel Fap Alfredo Mendivil Duarte,PE,2719.0
807,Mayor General FAP Armando Revoredo Iglesias Ai...,PE,2677.0
692,Licenciado Adolfo Lopez Mateos Intl,MEX,2581.0
